In [1]:
import streamlit as st
import os
from langchain.tools import DuckDuckGoSearchRun
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_agent_executor
from langchain_core.pydantic_v1 import BaseModel
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Annotated, Any, Dict, Optional, Sequence, TypedDict, List, Tuple
from dotenv import load_dotenv, dotenv_values 
import operator

load_dotenv()

True

In [2]:
langchain_api_key = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_PROJECT"] ="multi-agent"
os.environ['OPENAI_API_KEY'] = langchain_api_key

In [30]:
llm = ChatOpenAI(temperature=0, 
                          max_tokens=1024,
    model_name="gpt-4")

In [4]:
class DataScraper(BaseModel):
    """This Class is for Data Scraping"""
    
    code: str = Field(
        description = "Generate code based on the provided requirements"
    )
    
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate

data_scrape_prompt = ChatPromptTemplate.from_template(
    '''**Role**: You are an expert Python programmer with a focus on data manipulation and analysis. Your expertise extends to handling and processing data from various formats, including CSV files.

**Task**: Your task is to develop Python code that reads data from a CSV file located at './data.csv'. After reading the data, you must process it according to specific requirements to return a subset of the DataFrame. This task may involve filtering, aggregating, transforming, or any other form of data manipulation to achieve the desired output.

**Instructions**:
1. **Understand and Clarify**: Ensure you fully understand the requirement, including the structure of the data and the specific subset or transformation needed.
2. **Algorithm/Method Selection**: Choose the most efficient methods for reading the CSV file and processing the data. Consider using libraries such as pandas for efficient data manipulation.
3. **Pseudocode Creation**: Write down the steps to read the data from the CSV file, apply the necessary processing, and extract the required subset. This includes loading the data, inspecting its structure, applying specified transformations or filters, and selecting the relevant data to return.
4. **Code Generation**: Translate your pseudocode into executable Python code. Your code should be efficient, readable, and well-commented. Ensure to include necessary imports, read the CSV file into a DataFrame, perform the required data processing, and output the subset of data as text.

*REQUIREMENT*
{requirement}'''

)

coder = create_structured_output_runnable(
    DataScraper, llm, data_scrape_prompt
)

In [5]:
code_ = coder.invoke({'requirement': 'Get me all the data for the country of germany'})

Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')


Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')
Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')
Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')
Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API

In [6]:
print(code_.code)

import pandas as pd

# Read the CSV file
data = pd.read_csv('./data.csv')

# Filter data for the country of Germany
germany_data = data[data['Country'] == 'Germany']

# Display the subset of data
print(germany_data)


In [7]:
from io import StringIO
import sys
tmp = sys.stdout
my_result = StringIO()
sys.stdout = my_result
exec(code_.code)
sys.stdout = tmp

In [8]:
print(my_result.getvalue())

       TransactionNo       Date ProductNo                        ProductName  \
914           581494  12/9/2019     23084                 Rabbit Night Light   
915           581494  12/9/2019     21559  Strawberry Lunch Box With Cutlery   
916           581494  12/9/2019     21506           Fancy Font Birthday Card   
917           581494  12/9/2019     22716                 Card Circus Parade   
918           581494  12/9/2019     22556      Plasters In Tin Circus Parade   
...              ...        ...       ...                                ...   
536345       C536548  12/1/2018     22168       Organiser Wood Antique White   
536346       C536548  12/1/2018     21218             Red Spotty Biscuit Tin   
536347       C536548  12/1/2018     20957       Porcelain Hanging Bell Small   
536348       C536548  12/1/2018     22580       Advent Calendar Gingham Sack   
536349       C536548  12/1/2018     22767         Triple Photo Frame Cornice   

        Price  Quantity  CustomerNo  Co

In [31]:
class Insights(BaseModel):
    """Plan to follow in future"""
    
    insight: str = Field(
        description= "Insights based on the data we pass"
    )
    
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate

insight_gen_prompt = ChatPromptTemplate.from_template(
    '''
**Role**: You are an expert in data analysis, with a strong background in providing actionable business insights from complex datasets.
**Task**: Your objective is to analyze a dataset provided to you, which contains information in the form of rows and columns related to e-commerce transactions. Utilize your analytical skills to examine the data and extract valuable insights that can inform business decisions, such as inventory management, marketing strategies, and customer satisfaction initiatives, and after you do all this, return those insights.

**Instructions**:
1. **Data Exploration**: Start by understanding the dataset's structure, including the number of rows and columns and the types of data (numerical, categorical) it contains. Identify key columns that could provide valuable insights, such as product names, quantities sold, prices, and transaction countries.
2. **Data Processing and Analysis**: Clean and preprocess the data if necessary. Then, conduct an in-depth analysis to uncover trends, patterns, and anomalies. Focus on identifying top-selling products, revenue-generating items, sales distribution by country, average purchase values, and products with high return rates.
3. **Insight Extraction**: Based on your analysis, extract actionable insights that could help a business executive make informed decisions. This might include recommendations on stocking up certain products, identifying products that contribute most to the revenue, understanding market dynamics across different regions, and addressing issues related to product returns.
4. **Reporting and Visualization**: Present your findings in a clear and understandable manner, supported by data visualizations where appropriate. Your report should highlight key insights and suggest actions that could improve the business's performance.

*OUTPUT EXPECTATION*:
The expected outcome is a comprehensive report containing actionable insights derived from the dataset. These insights should inform business strategies such as which products to stock more of, how to enhance marketing efforts, understanding customer preferences, and how to reduce return rates. The insights should be presented in a manner that is easily understandable by business executives, focusing on improving profitability and customer satisfaction.

*REQUIREMENT*:
Provide a structured dataset containing e-commerce transactions
*Data*:
{data}
'''

)

insights_agent = create_structured_output_runnable(
    Insights, llm, insight_gen_prompt
)

In [32]:
insight_ = insights_agent.invoke({'data': my_result.getvalue()})

In [33]:
print(insight_.insight)

Based on the provided dataset, the following insights can be extracted:

1. **Top-Selling Products**: The 'Rabbit Night Light' and 'Strawberry Lunch Box With Cutlery' are among the top-selling products. These items should be kept in stock to meet customer demand.

2. **Revenue-Generating Items**: Products such as the 'Organiser Wood Antique White' and 'Triple Photo Frame Cornice' have high prices, indicating that they contribute significantly to the revenue. Marketing efforts can be enhanced for these products to increase sales.

3. **Sales Distribution by Country**: The dataset includes transactions from Germany. Understanding the market dynamics in this region can help tailor marketing strategies and inventory management.

4. **Average Purchase Values**: The average purchase value can be calculated by multiplying the quantity of each product sold by its price. This information can help identify which products are driving the most revenue.

5. **Products with High Return Rates**: Prod

# Graph Design

In [69]:
class AgentCoder(TypedDict):
    requirement: str
    data: str
    insights: str

In [70]:
def programmer(state):
    print("Entering the programmer")
    # print("\n===================")
    # print(state)
    # print("\n====================")
    requirement = state['requirement']
    code_ = coder.invoke({'requirement':requirement})
    tmp = sys.stdout
    data = StringIO()
    sys.stdout = data
    exec(code_.code)
    sys.stdout = tmp
    
    return {'data': data.getvalue()}
    
def insight_generator(state):
    print("Entering Insight Generator")
    # print("\n===================")
    # print(state)
    # print("\n====================")
    data = state['data']
    insight_ = insights_agent.invoke({'data': data})
    return {'insights': insight_.insight}
    
def decide_to_end(state):
    print(f'Entering in Decide to End')
    # print(state['insights'])
    if state['insights'] is not None:
        return 'end'
    else:
        return 'programmer'

In [71]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentCoder)


workflow.add_node('programmer', programmer)
workflow.add_node('insight_generator', insight_generator)

workflow.set_entry_point("programmer")
workflow.add_edge("programmer", "insight_generator")

workflow.add_conditional_edges(
    "insight_generator",
    decide_to_end,
    {
        "end": END,
        "programmer": "programmer",
    },
)

In [72]:
app = workflow.compile()

In [73]:
requirement = "Get me all the data for the country of france"

In [74]:
from langchain_core.messages import HumanMessage

config = {"recursion_limit": 50}
inputs = {"requirement": requirement}
running_dict = {}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        running_dict[k] = v
        if k != "__end__":
            print(v)
            print('----------'*20)

Entering the programmer
{'data': '       TransactionNo       Date ProductNo                     ProductName  \\\n1598          581587  12/9/2019     22631         Circus Parade Lunch Box   \n1599          581587  12/9/2019     22556   Plasters In Tin Circus Parade   \n1600          581587  12/9/2019     22555       Plasters In Tin Strongman   \n1601          581587  12/9/2019     22728       Alarm Clock Bakelike Pink   \n1602          581587  12/9/2019     22727        Alarm Clock Bakelike Red   \n...              ...        ...       ...                             ...   \n533303        536370  12/1/2018     22900  Set 2 Tea Towels I Love London   \n533304        536370  12/1/2018     21913  Vintage Seaside Jigsaw Puzzles   \n533305        536370  12/1/2018     22540       Mini Jigsaw Circus Parade   \n533306        536370  12/1/2018     22544            Mini Jigsaw Spaceboy   \n533307        536370  12/1/2018     22492          Mini Paint Set Vintage   \n\n        Price  Quantity  Cu

In [75]:
print(v['insights'])

Based on the provided dataset, the following insights can be derived:

1. **Top-Selling Products**: The 'Circus Parade Lunch Box', 'Plasters In Tin Circus Parade', 'Plasters In Tin Strongman', 'Alarm Clock Bakelike Pink', and 'Alarm Clock Bakelike Red' are among the top-selling products. These items should be kept in stock to meet customer demand.

2. **Revenue-Generating Items**: Products such as the 'Set 2 Tea Towels I Love London', 'Vintage Seaside Jigsaw Puzzles', 'Mini Jigsaw Circus Parade', 'Mini Jigsaw Spaceboy', and 'Mini Paint Set Vintage' have higher prices and are sold in larger quantities, contributing significantly to the revenue. Marketing efforts can be enhanced for these products to increase sales.

3. **Sales Distribution by Country**: The majority of the transactions are from France. This indicates a strong market presence in France, and strategies should be developed to maintain and increase market share in this region.

4. **Average Purchase Values**: The average pu